In [1]:
import numpy as np
import pandas as pd

train_df = pd.read_csv('dataset/train.csv')
test_df = pd.read_csv('dataset/test.csv')

In [2]:
from string import punctuation
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

import re


#list of stop words in 
stops = set(stopwords.words('english'))
#removing words which are not stop words from "stops" (key elements in emotional analysis)
not_stop = ["aren't","couldn't","didn't","doesn't","don't",
            "hadn't","hasn't","haven't","isn't","mightn't",
            "mustn't","needn't","no","nor","not","shan't",
            "shouldn't","wasn't","weren't","wouldn't"]
stops = [x for x in stops if x not in not_stop]
stemmer = SnowballStemmer('english')


def clean_tweet(tweet):
    """
    this method implements the data preprocessing step such as removel of stop words, stemming
    remove punctuation
    """
    # lower case letters
    tweet = tweet.lower()
    
    # remove links
    tweet = re.sub(r'^https?:\/\/.*[\r\n]*', '', tweet, flags=re.MULTILINE)
    
    # remove punctuation
    tweet = ''.join([c for c in tweet if c not in punctuation])
    
    # stopwords removal
    tweet = tweet.split() # split into individual words
    tweet = [w for w in tweet if not w in stops]
    
    # stemming of words
    tweet = [stemmer.stem(w) for w in tweet]
    
    return(' '.join(tweet))

In [3]:
train_df['clean_tweet'] = train_df['original_text'].apply(clean_tweet)
test_df['clean_tweet'] = test_df['original_text'].apply(clean_tweet)